In [ ]:
!pip install colab-xterm
!pip install whisper
!pip install pyngrok
!pip install ollama
!pip install pynvml
!pip install langchain_community
!pip install langchain_huggingface
!pip install langchain_ollama
!pip install chromadb
!pip install pypdf
!pip install tiktoken
!pip install -U openai-whisper
!choco install ffmpeg
!pip install setuptools-rust
!apt-get update
!apt-get install -y portaudio19-dev
!pip install sounddevice
!pip install flask flask_cors torch torchaudio transformers diffusers pyngrok sounddevice numpy whisper

/bin/bash: line 1: choco: command not found
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... D

In [ ]:
! whisper audio.flac audio.mp3 audio.wav --model large

100%|█████████████████████████████████████| 2.88G/2.88G [00:49<00:00, 62.4MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

# **Low GPU Use**

In [ ]:
from flask import Flask, jsonify, request
from flask_cors import CORS
import whisper
import torch
from diffusers import StableDiffusionPipeline
import base64
import io
from pyngrok import ngrok
import os

# Initialize Flask app
app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

# Initialize models with memory optimization
whisper_model = whisper.load_model("large")

# Optimize Stable Diffusion model
sd_model = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
sd_model.enable_attention_slicing()  # Reduce memory usage
sd_model.to("cuda")

# Home route to check server status
@app.route('/', methods=['GET'])
def home():
    return jsonify({"status": "Server is running"})

# Transcribe route with GET and POST methods
@app.route('/transcribe', methods=['GET', 'POST'])
def generate():
    if request.method == 'GET':
        return jsonify({"status": "Transcribe endpoint is ready"})

    try:
        # Get request data
        data = request.json

        # Check for text or audio input
        text_input = data.get('text', '')
        audio_data = data.get('audio', '')
        filename = data.get('filename', 'temp_audio.mp3')

        # Process audio if present
        if audio_data:
            # Decode base64 audio
            audio_bytes = base64.b64decode(audio_data)

            # Save to a temporary file
            with open(filename, 'wb') as f:
                f.write(audio_bytes)

            # Transcribe audio
            result = whisper_model.transcribe(filename)
            text = result["text"]
        else:
            text = text_input

        # Generate image if text is not empty
        if text.strip():
            # Generate image with reduced inference steps
            with torch.autocast("cuda"):
                with torch.inference_mode():
                    image = sd_model(
                        text,
                        num_inference_steps=20,  # Reduced from 30
                        guidance_scale=7.5  # Typical default
                    ).images[0]

            # Convert to base64
            buffered = io.BytesIO()
            image.save(buffered, format="PNG")
            img_str = base64.b64encode(buffered.getvalue()).decode()

            return jsonify({
                'text': text,
                'image': img_str
            })

        return jsonify({'error': 'No text detected'})

    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    ngrok.set_auth_token("2rsixIKt8bxTKPpcGLSt7tlzJJt_6aoJdomfaNKJHYv3kkCih")
    ngrok_tunnel = ngrok.connect(5000)
    print(f"Public URL: {ngrok_tunnel}")
    app.run(host='0.0.0.0', port=5000)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

100%|█████████████████████████████████████| 2.88G/2.88G [01:18<00:00, 39.2MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

INFO:werkzeug:127.0.0.1 - - [06/Feb/2025 07:38:57] "POST /transcribe HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Feb/2025 07:39:07] "OPTIONS /transcribe HTTP/1.1" 200 -


  0%|          | 0/20 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [06/Feb/2025 07:39:11] "POST /transcribe HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Feb/2025 07:39:21] "OPTIONS /transcribe HTTP/1.1" 200 -


  0%|          | 0/20 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [06/Feb/2025 07:39:26] "POST /transcribe HTTP/1.1" 200 -
